In [83]:
import numpy as np
from Crypto.Util.number import getPrime, inverse
from hashlib import sha256
from random import randint
from Crypto.PublicKey import RSA
import math as m

# Generate a keypair for both the sender and receiver

In [84]:
# sender_keypair = RSA.generate(2048) #recommended 2048 key length
# receiver_keypair = RSA.generate(2048)

# pwd = b'secure_password'
# with open("sender_privkey.pem", "wb") as f:
#     data = sender_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [85]:
# pwd = b'secure_password_receiver'
# with open("receiver_privkey.pem", "wb") as f:
#     data = receiver_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [86]:
# with open("receiver_pubkey.pem", "wb") as f:
#     data = receiver_keypair.public_key().export_key()
#     f.write(data)

In [87]:
# with open("sender_pubkey.pem", "wb") as f:
#     data = sender_keypair.public_key().export_key()
#     f.write(data)

# Message (m) and Encryption -> M

In [88]:
msg = 'secret message'.encode('utf-8')
msg

b'secret message'

# Key generation

In [89]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

In [90]:
sender_keypair = generate_keypair()
receiver_keypair = generate_keypair()
print(sender_keypair, '\n', receiver_keypair)

((13874686323787543361091696000387236446188902559048182992734722888776856560982370375294260376125850384500292496243997679200376812917786306734144467502546486894558519819106826301668401887331705179428218952007150300633487449796372524077560263194506732140363628212592780095666724352461745255856321670632364708301610595612255580881476713091757132129260136509733315607675797664824511321159343453074376949246094869597011260705456957770889549797109310717272897657061901295347926732923948003699028844785086632677960669977428561707270931892848986549983132877221044049358065434628027492479658781095974828965131704017043039439503, 5928313193013404403700754606864666090583056722247360017529973971509585147968532431005491521832783008670751438608720819522203990992484518021035819260964184124128128302098817467779479525138184202973532895529969985617750203141159568840649055313085687789381276496989921897825384930499192696441305498509751385771316041124168030139358100858556230310306882976509121454548650920852114512

x -> encoded plaintext
y -> encrypted plaintext
h -> hashed x

# Encoding Message

In [91]:
msg = 'secret message'.encode('utf-8')
x = int.from_bytes(msg, 'big')
x

2340509926146499719798124187838309

In [92]:
hacker_msg = 'not secret'.encode('utf-8')
x_hack = int.from_bytes(hacker_msg, 'big')
x_hack

521516269522829685450100

# Encryption

In [93]:
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n)
y

1844533313139321824226653650811985318782945628865538471097598424213983243966275771346151131941896201128104633437903350882418029969595368759349626259605922954084274069689273476649757589751214359690914682754053153131217016591738976018989346499844987801602596774444677870429981106863110930610546003484331568515027551162107480773280884940364346265826949246920739777831188612872459910617115175481988979345563291292815233608267978869620430162215494727199094641633535140510825583862415402168587708556686918584233773643598174753429390381855404086957062824617077314713830293315095625000814576443737946186309165058884622143166

In [94]:
y_hack = pow(x_hack, e, n)
y_hack

6945779932734575239917278514443242751849346430436073077953020390195427174388207445513040132381177628856131544829207296320642962919698385600538577192971509492133293240628918212553746033103126439484494411657484816159579969507042865670922971526404080954509308512356449037712603562931542245937907175326335089285287719185222564275747999885223422933876773127715565913671423097346517973663985151343206001455100209865008457358526105362170246205192735037832548301707398070130709071314363118150005783178531412100196051781668132586383828809082731548054444893508749346789750081211430547612423940884224177297589537714461724500668

# Hash

In [95]:
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')
h

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Signing

In [96]:
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n)
sign 

729609642748714020919367270356076965203063172431559256470744819111912585936620274820521493503569055176462010434807742459874686757259560050566163920143348340842064304787016582704769172856541757903266003593999620984680207767733508910520223395234516806019394081624959871415040588519564883803875464484767495185604016218034975886462587009837780389357969836648730413646178025609451132359671736194877986960018006511738253794296213603622258327505824799566493427327639795888153141078504539791717703838271325540849209955652098957548422330861121331438658646014056045075195225257474986487422068191949393409241924379614453961060

# Decryption of hash (sign) with Sender pub key

In [97]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)
decrypt_to_hash

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Decryption of message

In [98]:
n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)
decrypt_to_msg

2340509926146499719798124187838309

# Hash verification 

In [99]:
msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
print(int.from_bytes(final_h.digest()) == decrypt_to_hash)
#print(final_h.digest() == decrypt_to_hash)

True


# Get plaintext

In [100]:
decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

'secret message'

In [101]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

# Generating keypairs
sender_keypair = generate_keypair() #generate sender keypair
receiver_keypair = generate_keypair() #generate receiver keypair
print(sender_keypair, '\n', receiver_keypair)

#Define message x
msg = 'secret message'.encode('utf-8') #secret message
x = int.from_bytes(msg, 'big') #turn bytes to integer

# Encrypt message x with receiver public key -> ciphertext y
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n) 

# Compute hash
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')

#Signing 
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n) #signing hash with sender private key

((14475856830261544067947428119414447155984451687003678122026832208690254364514264722290571531569495419711535117417167043682701929254210512513037333278882640215153547514948763018187147499362628365214087263429668842793406268769379854735785711932375835870357398001562263355049543989009079988186709199267424938220839028862772483026728563884497233321085159216571024398840812240234796027054838874976419735235580273552143880897364989350071632124715371973634421774375220009611928383741000610749398892467980788942504163472492879131051913078955142856750909299677607529546171764835954451387790299574292125176389545133839863375477, 1160561791565537470385835164639702376100288159423206719049003909239004533297571079504319755214911677403209571255629856404971315168992043682614110127773943111723479044662522974004419188459556733456181524620278205449596274831725522746003665359393035219656530984877837340128701680123539538294423774868640694860186073115938357014843106528018012351286633599366415101899770369575396145

In [102]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)

n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)

msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
final_h_int = int.from_bytes(final_h.digest(), 'big')  

valid_signature = decrypt_to_hash == final_h_int
print("Signature valid?", valid_signature)

decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

Signature valid? True


'secret message'

# 3DES

In [103]:
from Crypto.Cipher import DES3, AES
from Crypto.Random import get_random_bytes

def generate_3des_key():
    while True:
        key = get_random_bytes(24)
        try:
            key = DES3.adjust_key_parity(key)
        except ValueError:
            continue
        
        k1, k2, k3 = key[:8], key[8:16], key[16:]
        
        if k1 != k2 and k2 != k3 and k1 != k3:
            return key, (k1, k2, k3)

key, (k1, k2, k3) = generate_3des_key()

print("3DES key (hex):", key.hex())
print("K1:", k1.hex())
print("K2:", k2.hex())
print("K3:", k3.hex())


3DES key (hex): 928c04a792b0ef439bced37c760b7ce51502a46737f74a1c
K1: 928c04a792b0ef43
K2: 9bced37c760b7ce5
K3: 1502a46737f74a1c


In [104]:
from base64 import b64encode, b64decode
from Crypto.Util.Padding import pad, unpad

def encypt_3DES(plaintext, key):
    cipher = DES3.new(key, DES3.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(plaintext, DES3.block_size)) 
    initial_vector = b64encode(cipher.iv).decode('utf-8') #initialization vector -> probabilistic method
    ct = b64encode(ct_bytes).decode('utf-8') #cipher text
    return initial_vector, ct

In [105]:
data = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit,' \
'sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. ' \
'Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ' \
'ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate ' \
'velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, ' \
'sunt in culpa qui officia deserunt mollit anim id est laborum.'.encode('utf_8')
iv, ct = encypt_3DES(plaintext=data, key=key)
print(ct)

9DylsPct8KL5fkO5jgHudm+PJI1wtZ8aat3HijHL3RkkXkXNuiPOcm7UqU85gtIFTgcf/CVTQBQxqQ4O8fuDnqUY5ssB9kZUVw8k0T8S3qGH4BfzOOaTU7sppbHiqujOU0mPADnm2YdmtxNdzColJZL+LQ8FMiBRhYLmHs3XU8FSd/U5fiGyapP079QcUdrs9idqP10NNsKtW+j+PHkTVmE9SmvHDBV32A+u3Dk8L07ez9flCHNYLrcL6BKz14MevbIZ4mju8bWlZYh4UUBKYOST9c+XrgZ3AwcsHFdQ1vvEtkAmGWMwBJJuPqAwovLVuc8NeUjNEbqov1fOC4oMIY/ZJzNyQlWe+mVrc/OpC+38dpvN8A215pY77vyp2xiJ2mfk2SVUHXQ48IazkR2RnrADJLZ5xOKQ8yySYN/LbR0qmpGTn45nZjpqophSItvsUpMm1/vRlXbmPCPHBt0ihIzIXuoF3GV5jqlyzDyed6+hd2/dHqFk/Np7Dx/DHPW33+DmWauy54JmwRj6buXUcF7BZvNxfMFX+e+TxT8tIswzTjOg1L4lx32IklDj4fcLgnlui3uubBulKKIqWu3srA==


In [106]:
def decrypt_3DES(iv, ct, key):
    try:
        cipher = DES3.new(key, DES3.MODE_CBC, iv= b64decode(iv))
        pt = unpad(cipher.decrypt(b64decode(ct)), DES3.block_size)
        print("The message was: ", pt)
    except (ValueError, KeyError):
        print("Incorrect decryption")

In [107]:
decrypt_3DES(iv, ct, key)

The message was:  b'Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'


# AES encryption

In [108]:
from Crypto.Cipher import AES
from Crypto.Util import Counter

nonce = get_random_bytes(8)    # 8-byte nonce (64 bits)
key = get_random_bytes(32)     # AES-256 key

ctr_encrypt = Counter.new(64, prefix=nonce)   # 64-bit counter, prefixed with nonce
aes_cipher_encrypt = AES.new(key, AES.MODE_CTR, counter=ctr_encrypt)
ciphertext = aes_cipher_encrypt.encrypt(data)

encoded_ciphertext = b64encode(ciphertext).decode("utf-8")
print("Ciphertext (Base64):", encoded_ciphertext)



Ciphertext (Base64): 2KG0EVM01lofNlr14JcZCy8wsfGGo5icavQydA3dnvtEmOVvxL60/VCOS46F7sCe+GkOViW3tftKL52edX/hDdzVixKvOjWmiQl1B2IEOKbwOvBe+PyiYf9i1S+73N4OVG1ohiOr6BqdC6R0gjmXVqHnNmt0qwaHCW/Bo8KSxuWxcoxkreAvsgmyx+BwmoI1Rpci8a+/QC9D8psBloP6kK0c7Pw2VtOjyPF/q+lY5CZTPhJHbtXNYCc48B/iIx5uKLIXKc99jZyPpf4CGNYGtkyCtKGblTZKPAZL/46cnMftRIZPD5qqbJwfyinY19OXdUX56y4GnE3ygqZCTuE8+3uGiAlynN8WPlTA17pa/AaXrmsL1obTcjTPH8SkJ7Kc+wB/5hBrWPGLVlcymvvhbOM/595X1Ne/r2ZX8JDqBRgig28tt5rv9oxYpAsENVh3gLx8LZB3hgcIfwYV7WDSvVml/b7geyQ43WPgxnq3APOBxN3JXCi2ORYf5WqgVt8TOBF9yM02o3YOejz1f2DfAyzO/7U3wgJed7vYtX9vUr2rgQGk+Fe6fh8KOPZZsP6V/mJ+lJu5rgC1bPRM


## AES Decryption

In [109]:

ctr_dec = Counter.new(64, prefix=nonce)
aes_decipher = AES.new(key, AES.MODE_CTR, counter=ctr_dec)
decrypted_bytes = aes_decipher.decrypt(b64decode(encoded_ciphertext))

decrypted_text = decrypted_bytes.decode('utf-8')
print("Decrypted (readable):", decrypted_text)


Decrypted (readable): Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.


## Lesson 7 Python Qs

1. Why must you follow the algorithm steps exactly as prescribed? What happens if you skip some step or apply it in different order?
- ????
2. How did you securely obtain the keys for encryption? How would you exchange these keys to the receiver of you message? How long should the key be considered valid (expiration)? How can you estimate/decide when a key is no longer secure enough?
- For DES, we check if k1 not equal to k2 and k3 
- For AES, it is just a random 256 bit key
- We would exchange using public key encryption like RSA
- 1 message session??? like every reply?? verify with source
- Compute moore's law yaaa research lg

3. Why do we use tripple DES and not a double DES? Is double DES stronger than single DES? Would quadrouple DES be safer to use than tripple DES? Explain.
- if double DES, the key length is only 80 bits effective. yes? because key is longer but idk about computer power HRS SEARCH

4. Why is it advised to have a plain text longer than block size? You encrypted a lorem impsum, which is a clear text with a pattern. Why does your encrypted cipher text has no visible patterns in it? Compare what extra steps in AES are taken to ensure that the plain text pattern is concealed (vs. simple substitution cipher).

BACA LG ANJGGG

5. Can encryption of two different texts result in the same cipher text? How this can be done and what implications does it have on security?

NO?? bcs of iv?? or because of ctr block???? GATAUU WOI


